In [1]:
# Dependencies
import pandas as pd
import json
import requests
import datetime as dt

# import sqlalchemy
# from sqlalchemy.ext.automap import automap_base
# from sqlalchemy.orm import Session
# from sqlalchemy import create_engine, func, inspect
# import sqlite3

In [2]:
with open('Resources/ColoCitiesDataClean.csv') as f:
    print(f)

<_io.TextIOWrapper name='Resources/ColoCitiesDataClean.csv' mode='r' encoding='cp1252'>


In [3]:
latlon_df = pd.read_csv("Resources/ColoCitiesDataClean.csv", encoding='cp1252')
latlon_df.head()

,Place Name,lat,lon,Population,Area
0,Aguilar,37.403611,-104.655000,480,1.016
1,Akron,40.163333,-103.220278,"1,723",7.180
2,Alamosa,37.475278,-105.876944,"9,591",19.871
3,Alma,39.285833,-106.066944,313,1.129
4,Antonito,37.076111,-106.010278,747,1.105


In [4]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/air_pollution?"
api_key= "90d059c74bd13bcc4087b66e982c1bf1"
lat=37.40361111
lon=-104.655

#Build URL
query_url = f"{url}lat={lat}&lon={lon}&appid={api_key}"

#Query API
air_response = requests.get(query_url).json()

#Test API response
print(air_response)

{'coord': {'lon': -104.655, 'lat': 37.4036}, 'list': [{'main': {'aqi': 2}, 'components': {'co': 188.59, 'no': 0, 'no2': 1, 'o3': 82.97, 'so2': 0.11, 'pm2_5': 6.76, 'pm10': 9, 'nh3': 1.16}, 'dt': 1626753600}]}


In [5]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/air_pollution?"
api_key= "90d059c74bd13bcc4087b66e982c1bf1"

# Counter to sort through response
number = 0
group = 1

# Create DataFrame
## Update PM names to remove spaces
columns= ["City","Date", "lat", "lon", "AQI", "CO", "NO", "NO2", "O3", "SO2", "PM25", "PM10", "NH3"]
air_df = pd.DataFrame(columns=columns)

for index, row in latlon_df.iterrows():
    
    lat = row["lat"]
    lon = row["lon"]
    #print(lat)
    #print(lon)
    query_url = f"{url}lat={lat}&lon={lon}&appid={api_key}"
    air_response = requests.get(query_url).json()
 
    try:
        print(f"Processing Record {number}  of Group {group} | {lat}{lon}.")
        
        print (query_url) 
    
        air_api_values= {
            "City" : row["Place Name"],
            "Population": row["Population"],
            "Date" : air_response["list"][0]["dt"],  
             "lon" : air_response["coord"]["lon"],
             "lat" : air_response["coord"]["lat"],
             "AQI" : air_response["list"][0]["main"]["aqi"],
             "CO" : air_response["list"][0]["components"]["co"],  
             "NO" : air_response["list"][0]["components"]["no"],
             "NO2" : air_response["list"][0]["components"]["no2"],  
             "O3" : air_response["list"][0]["components"]["o3"],
             "SO2" : air_response["list"][0]["components"]["so2"],
            #Update names of PM so they don't have spaces
             "PM25" : air_response["list"][0]["components"]["pm2_5"],
             "PM10" : air_response["list"][0]["components"]["pm10"],
             "NH3" : air_response["list"][0]["components"]["nh3"]
           }
        air_df= air_df.append(air_api_values, ignore_index=True)
        
    except:
        print(f"not working")
        pass
        
    
    # To save money on API calls
    number = number + 1
    if number == 50:
        number= 0
        group = group + 1


Processing Record 0  of Group 1 | 37.40361111-104.655.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.40361111&lon=-104.655&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 1  of Group 1 | 40.16333333-103.22027779999999.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.16333333&lon=-103.22027779999999&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 2  of Group 1 | 37.47527778-105.87694440000001.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.47527778&lon=-105.87694440000001&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 3  of Group 1 | 39.28583333-106.06694440000001.
http://api.openweathermap.org/data/2.5/air_pollution?lat=39.28583333&lon=-106.06694440000001&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 4  of Group 1 | 37.07611111-106.0102778.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.07611111&lon=-106.0102778&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 5  of Group 1 | 39.28444444-

Processing Record 44  of Group 1 | 38.81916667-102.35194440000001.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.81916667&lon=-102.35194440000001&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 45  of Group 1 | 38.36222222-105.1422222.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.36222222&lon=-105.1422222&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 46  of Group 1 | 37.14416667-104.6216667.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.14416667&lon=-104.6216667&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 47  of Group 1 | 39.24-107.96361110000001.
http://api.openweathermap.org/data/2.5/air_pollution?lat=39.24&lon=-107.96361110000001&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 48  of Group 1 | 38.8675-104.7605556.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.8675&lon=-104.7605556&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 49  of Group 1 | 39.59944444-105.03527779999999.


Processing Record 38  of Group 2 | 40.5475-105.06555559999998.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.5475&lon=-105.06555559999998&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 39  of Group 2 | 40.08277778-104.8069444.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.08277778&lon=-104.8069444&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 40  of Group 2 | 40.25361111-103.7902778.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.25361111&lon=-103.7902778&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 41  of Group 2 | 38.68833333-104.6827778.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.68833333&lon=-104.6827778&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 42  of Group 2 | 38.13055556-104.02638890000001.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.13055556&lon=-104.02638890000001&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 43  of Group 2 | 39.58805556-104.7858

Processing Record 32  of Group 3 | 40.985-102.2627778.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.985&lon=-102.2627778&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 33  of Group 3 | 40.13277778-104.53805559999999.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.13277778&lon=-104.53805559999999&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 34  of Group 3 | 40.39083333-104.5794444.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.39083333&lon=-104.5794444&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 35  of Group 3 | 37.24694444-103.3533333.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.24694444&lon=-103.3533333&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 36  of Group 3 | 39.34361111-104.45777779999999.
http://api.openweathermap.org/data/2.5/air_pollution?lat=39.34361111&lon=-104.45777779999999&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 37  of Group 3 | 38.76277778-102.795277

Processing Record 26  of Group 4 | 38.90833333-106.96055559999999.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.90833333&lon=-106.96055559999999&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 27  of Group 4 | 39.77472222-105.05666670000001.
http://api.openweathermap.org/data/2.5/air_pollution?lat=39.77472222&lon=-105.05666670000001&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 28  of Group 4 | 37.9325-107.8577778.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.9325&lon=-107.8577778&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 29  of Group 4 | 38.21833333-108.56666670000001.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.21833333&lon=-108.56666670000001&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 30  of Group 4 | 39.96333333-105.5072222.
http://api.openweathermap.org/data/2.5/air_pollution?lat=39.96333333&lon=-105.5072222&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 31  of Group 4 | 39.577

Processing Record 20  of Group 5 | 38.08638889-106.1411111.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.08638889&lon=-106.1411111&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 21  of Group 5 | 38.53-105.9977778.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.53&lon=-105.9977778&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 22  of Group 5 | 37.2025-105.4225.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.2025&lon=-105.4225&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 23  of Group 5 | 37.2575-105.90083329999999.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.2575&lon=-105.90083329999999&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 24  of Group 5 | 37.99472222-108.0022222.
http://api.openweathermap.org/data/2.5/air_pollution?lat=37.99472222&lon=-108.0022222&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 25  of Group 5 | 40.93555556-102.5255556.
http://api.openweathermap.org/data/

Processing Record 14  of Group 6 | 38.15527778-102.7191667.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.15527778&lon=-102.7191667&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 15  of Group 6 | 38.38388889-105.1711111.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.38388889&lon=-105.1711111&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 16  of Group 6 | 40.46888889-104.92.
http://api.openweathermap.org/data/2.5/air_pollution?lat=40.46888889&lon=-104.92&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 17  of Group 6 | 39.87861111-105.78277779999999.
http://api.openweathermap.org/data/2.5/air_pollution?lat=39.87861111&lon=-105.78277779999999&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 18  of Group 6 | 38.99861111-105.05944440000002.
http://api.openweathermap.org/data/2.5/air_pollution?lat=38.99861111&lon=-105.05944440000002&appid=90d059c74bd13bcc4087b66e982c1bf1
Processing Record 19  of Group 6 | 40.08-102.2283333.
htt

In [6]:
print(air_df)

              City        Date      lat       lon AQI      CO NO   NO2  \
0          Aguilar  1626753600  37.4036 -104.6550   2  188.59  0     1   
1            Akron  1626753600  40.1633 -103.2203   2  198.60  0  0.72   
2          Alamosa  1626753600  37.4753 -105.8769   2  188.59  0  0.87   
3             Alma  1626753600  39.2858 -106.0669   2  168.56  0  1.16   
4         Antonito  1626753600  37.0761 -106.0103   1  171.90  0   0.7   
..             ...         ...      ...       ...  ..     ... ..   ...   
267    Winter Park  1626753600  39.8786 -105.7828   2  183.58  0  2.14   
268  Woodland Park  1626753600  38.9986 -105.0594   1  190.26  0  1.89   
269           Wray  1626753600  40.0800 -102.2283   1  208.62  0  0.84   
270          Yampa  1626753600  40.1531 -106.9086   2  176.91  0  0.81   
271           Yuma  1626753600  40.1236 -102.7161   1  205.28  0  0.88   

         O3   SO2   PM25   PM10   NH3 Population  
0     82.97  0.11   6.76      9  1.16        480  
1     81.

In [7]:
# obtain datetime series:
datetimes = pd.to_datetime(air_df['Date'], unit='s')

# assign your new columns
air_df['Year'] = datetimes.dt.year
air_df['Month'] = datetimes.dt.month
air_df['Day'] = datetimes.dt.day
air_df['Hour'] = datetimes.dt.hour
air_df['Minute'] = datetimes.dt.minute

In [8]:
#Clean city names
air_df["City"] = air_df['City'].str.split('[').str[0]
air_df.head()

,City,Date,lat,lon,AQI,CO,NO,NO2,O3,SO2,PM25,PM10,NH3,Population,Year,Month,Day,Hour,Minute
0,Aguilar,1626753600,37.4036,-104.6550,2,188.59,0,1,82.97,0.11,6.76,9,1.16,480,2021,7,20,4,0
1,Akron,1626753600,40.1633,-103.2203,2,198.60,0,0.72,81.54,0.12,8.42,9.13,1.33,"1,723",2021,7,20,4,0
2,Alamosa,1626753600,37.4753,-105.8769,2,188.59,0,0.87,87.98,0.16,9.65,11.28,0.94,"9,591",2021,7,20,4,0
3,Alma,1626753600,39.2858,-106.0669,2,168.56,0,1.16,97.28,0.49,13.97,15.31,0.52,313,2021,7,20,4,0
4,Antonito,1626753600,37.0761,-106.0103,1,171.90,0,0.7,78.68,0.12,7.54,8.75,1.14,747,2021,7,20,4,0


In [9]:
#Export to CSV
air_df.to_csv('Resources/data/air_api_data.csv')
#Export DF to html file
air_df.to_html('Resources/webpages/air_api_data.html')

In [10]:
import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.orm import Session

import sqlite3

# from sqlalchemy.ext.automap import automap_base
# from sqlalchemy.orm import Session
# import sqlite3

In [11]:
airquality_df = pd.read_csv("./Resources/air_api_data.csv")
airquality_df

,City,Date,lat,lon,AQI,CO,NO,NO2,O3,SO2,PM25,PM10,NH3,Population,Year,Month,Day,Hour,Minute
0,Aguilar,1626742800,37.4036,-104.6550,2,193.60,0.04,0.59,91.55,0.13,7.49,9.81,0.88,480,2021,7,20,1,0
1,Akron,1626742800,40.1633,-103.2203,2,193.60,0.03,0.56,89.41,0.16,8.52,9.54,1.01,"1,723",2021,7,20,1,0
2,Alamosa,1626742800,37.4753,-105.8769,2,176.91,0.03,0.61,107.29,0.28,8.46,9.73,0.67,"9,591",2021,7,20,1,0
3,Alma,1626742800,39.2858,-106.0669,2,175.24,0.05,0.66,115.87,1.15,14.75,16.21,0.56,313,2021,7,20,1,0
4,Antonito,1626742800,37.0761,-106.0103,2,173.57,0.02,0.58,97.28,0.19,6.86,7.84,0.74,747,2021,7,20,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,Winter Park,1626742800,39.8786,-105.7828,3,181.91,0.05,0.63,127.32,0.78,14.32,15.99,0.78,"1,090",2021,7,20,1,0
268,Woodland Park,1626742800,38.9986,-105.0594,2,181.91,0.06,0.75,98.71,0.39,8.91,10.15,0.74,"7,885",2021,7,20,1,0
269,Wray,1626742800,40.0800,-102.2283,2,205.28,0.03,0.53,85.12,0.12,8.10,8.72,1.17,"2,349",2021,7,20,1,0
270,Yampa,1626742800,40.1531,-106.9086,2,170.23,0.07,0.84,108.72,1.09,11.17,12.82,0.48,464,2021,7,20,1,0


In [12]:
engine = create_engine('sqlite:///Resources/air.sqlite')
conn = engine.connect()

In [13]:
air_df.to_sql('airquality', conn, if_exists='replace', index=False)

In [14]:
inspector=inspect(engine)
inspector.get_table_names()

['airquality']

In [15]:
conn.execute('pragma table_info(airquality);').fetchall()

[(0, 'City', 'TEXT', 0, None, 0),
 (1, 'Date', 'BIGINT', 0, None, 0),
 (2, 'lat', 'FLOAT', 0, None, 0),
 (3, 'lon', 'FLOAT', 0, None, 0),
 (4, 'AQI', 'BIGINT', 0, None, 0),
 (5, 'CO', 'FLOAT', 0, None, 0),
 (6, 'NO', 'BIGINT', 0, None, 0),
 (7, 'NO2', 'TEXT', 0, None, 0),
 (8, 'O3', 'FLOAT', 0, None, 0),
 (9, 'SO2', 'FLOAT', 0, None, 0),
 (10, 'PM25', 'FLOAT', 0, None, 0),
 (11, 'PM10', 'TEXT', 0, None, 0),
 (12, 'NH3', 'FLOAT', 0, None, 0),
 (13, 'Population', 'TEXT', 0, None, 0),
 (14, 'Year', 'BIGINT', 0, None, 0),
 (15, 'Month', 'BIGINT', 0, None, 0),
 (16, 'Day', 'BIGINT', 0, None, 0),
 (17, 'Hour', 'BIGINT', 0, None, 0),
 (18, 'Minute', 'BIGINT', 0, None, 0)]

In [16]:
conn.execute('select * from airquality limit 10').fetchall()

[('Aguilar', 1626753600, 37.4036, -104.655, 2, 188.59, 0, '1', 82.97, 0.11, 6.76, '9', 1.16, '480', 2021, 7, 20, 4, 0),
 ('Akron', 1626753600, 40.1633, -103.2203, 2, 198.6, 0, '0.72', 81.54, 0.12, 8.42, '9.13', 1.33, '1,723', 2021, 7, 20, 4, 0),
 ('Alamosa', 1626753600, 37.4753, -105.8769, 2, 188.59, 0, '0.87', 87.98, 0.16, 9.65, '11.28', 0.94, '9,591', 2021, 7, 20, 4, 0),
 ('Alma', 1626753600, 39.2858, -106.0669, 2, 168.56, 0, '1.16', 97.28, 0.49, 13.97, '15.31', 0.52, '313', 2021, 7, 20, 4, 0),
 ('Antonito', 1626753600, 37.0761, -106.0103, 1, 171.9, 0, '0.7', 78.68, 0.12, 7.54, '8.75', 1.14, '747', 2021, 7, 20, 4, 0),
 ('Arriba', 1626753600, 39.2844, -103.2742, 1, 195.27, 0, '1.03', 72.24, 0.11, 7.66, '8.45', 1.27, '207', 2021, 7, 20, 4, 0),
 ('Arvada', 1626753600, 39.8319, -105.1511, 2, 247.0, 0, '15.77', 69.38, 1.06, 13.6, '15.63', 1.73, '121,272', 2021, 7, 20, 4, 0),
 ('Aspen', 1626753600, 39.195, -106.8367, 2, 181.91, 0, '0.73', 60.08, 0.26, 11.45, '12.77', 1.9, '7,401', 2021, 7,

In [17]:
conn.close()